## This project is about enriching bookshelf dataset

Goodreads API<br>
https://www.goodreads.com/api

OpenLibrary API<br>
https://openlibrary.org/dev/docs/api/books

Lets read the dataset

In [34]:
!wget -O MyBooks.csv https://www.dropbox.com/s/ic8g9euhj4bh885/MyBooks.csv?dl=0

--2020-07-09 17:44:43--  https://www.dropbox.com/s/ic8g9euhj4bh885/MyBooks.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ic8g9euhj4bh885/MyBooks.csv [following]
--2020-07-09 17:44:43--  https://www.dropbox.com/s/raw/ic8g9euhj4bh885/MyBooks.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc74ade69784be7043cf6b82922d.dl.dropboxusercontent.com/cd/0/inline/A7O9-f385PNsS3lTQtCTyxCYavaEl9aAk2mLhHLW9Akx6MqjcKNDZx8jq9s1l47VSQmNFyScLfbJqwdltzLer9Thvn4s9JqWFZSugHfUC3Zsl3rJ4s-d3kbwGVL6La3ptnA/file# [following]
--2020-07-09 17:44:44--  https://uc74ade69784be7043cf6b82922d.dl.dropboxusercontent.com/cd/0/inline/A7O9-f385PNsS3lTQtCTyxCYavaEl9aAk2mLhHLW9Akx6MqjcKNDZx8jq9s1l47VSQmNFyScLfbJqwdltzLer9Thvn4s9JqWFZSugHfUC3Zsl3rJ

In [2]:
# install some packages
!pip install rauth

print("done..")

  Stored in directory: /home/dsxuser/.cache/pip/wheels/7b/94/5d/81afc278dd5da884a0002563dc4b0fe85f9067a5a40f76f858
Successfully built rauth
done..


In [3]:
# import some libs
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import rauth
import requests
import json

from rauth.service import OAuth1Service, OAuth1Session

print("done..")

done..


In [35]:
# inirial dataset

df = pd.read_csv("MyBooks.csv", error_bad_lines=False)
df

,ISBN Code,Title,Author,Published Date,Memo,Page count,Language,Country,Desciption,Image(URL)
0,9789663437354,Вінні Пух i всi всi всі,Александр А. Мiлн,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9789664620397,Іграшковi казки,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9786176951223,Казки про принцес,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9786177084180,Сказки для малышей,NaN,2016,NaN,61.0,ru,US,NaN,NaN
4,9789664625453,Новорічні казки,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,9789667047856,Найновіші пригоди Колька Колючки та Косі Вуханя,Всеволод Нестайко,4/9/09,NaN,160.0,uk,US,NaN,http://books.google.com/books/content?id=WtP9C...
6,966716134X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,9789661200394,Незвичайні історії про Іванка,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9789663190792,Королівство рожевих снів: Казки та легенди,Круковська Галина,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9663190795,Королівство рожевих снів: Казки та легенди,Круковська Галина,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Goodreads enrishment attempt

In [12]:
# lets define sone useful stuff
# https://www.goodreads.com/api/keys
CONSUMER_KEY = "__CONSUMER_KEY__"
CONSUMER_SECRET = "__CONSUMER_SECRET__"

goodreads = OAuth1Service(
    consumer_key = CONSUMER_KEY,
    consumer_secret = CONSUMER_SECRET,
    name = 'goodreads',
    request_token_url = 'https://www.goodreads.com/oauth/request_token',
    authorize_url = 'https://www.goodreads.com/oauth/authorize',
    access_token_url = 'https://www.goodreads.com/oauth/access_token',
    base_url = 'https://www.goodreads.com/'
    )

# head_auth=True is important here; this doesn't work with oauth2 for some reason
request_token, request_token_secret = goodreads.get_request_token(header_auth=True)
authorize_url = goodreads.get_authorize_url(request_token)
print("Visit this URL in your browser: ", authorize_url)

Visit this URL in your browser:  https://www.goodreads.com/oauth/authorize?oauth_token=ukeYZpBAhgYFxWwwBog


In [13]:
session = goodreads.get_auth_session(request_token, request_token_secret)
request_url = "https://www.goodreads.com/book/isbn/{}?key={}"

In [37]:
df.columns

Index(['ISBN Code', 'Title', 'Author', 'Published Date', 'Memo', 'Page count',
       'Language', 'Country', 'Desciption', 'Image(URL)'],
      dtype='object')

In [16]:
# debug ..

responce = session.get(request_url.format(9789667047856, CONSUMER_KEY))
root = ET.fromstring(responce.content)
tag_indexes = [1,7,8,9,10,13,15,18,19,20,21,22,23,24]
for index in tag_indexes:
    print(root[1][index].tag + ": " + str(root[1][index].text))

title: Найновіші пригоди їжачка Колька Колючки та зайчика Косі Вуханя
country_code: US
image_url: https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1439593496l/26101437._SX98_.jpg
small_image_url: https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1439593496l/26101437._SX50_.jpg
publication_year: 2009
publisher: А-БА-БА-ГА-ЛА-МА-ГА
is_ebook: false
average_rating: 4.55
num_pages: 160
format: Hardcover
edition_information: None
ratings_count: 20
text_reviews_count: 2
url: https://www.goodreads.com/book/show/26101437


In [40]:
count = 0

EMPTY = "    Empty"
NOT_EMPTY = "Not Empty"
FOUND = "Found"
NOT_FOUND = "Not Found"

for index, row in df.iterrows():
    responce = session.get(request_url.format(row["ISBN Code"], CONSUMER_KEY))
    root = ET.fromstring(responce.content)

    old_title = EMPTY if row["Title"] == None else NOT_EMPTY
    
    try:
        new_title = str(root[1][1].text)
        # uncomment this to track progress
        # print(old_title, " => ", FOUND)
        if old_title != NOT_EMPTY:
            # do enrichment
            count += 1
            row["Title"] = root[1][1].text
            row["Page count"] = root[1][19].text
            row["Country"] = root[1][7].text
            row["Image(URL)"] = root[1][8].text
    except:
        # uncomment this to track progress
        # print(old_title, " => ", NOT_FOUND)
        pass

print("Ok, we found {} useful updates in goodreads database".format(count))

Ok, we found 0 useful updates in goodreads database


In [42]:
df[["ISBN Code","Title","Author","Published Date","Page count","Language","Country","Image(URL)"]]

,ISBN Code,Title,Author,Published Date,Page count,Language,Country,Image(URL)
0,9789663437354,Вінні Пух i всi всi всі,Александр А. Мiлн,NaN,NaN,NaN,NaN,NaN
1,9789664620397,Іграшковi казки,NaN,NaN,NaN,NaN,NaN,NaN
2,9786176951223,Казки про принцес,NaN,NaN,NaN,NaN,NaN,NaN
3,9786177084180,Сказки для малышей,NaN,2016,61.0,ru,US,NaN
4,9789664625453,Новорічні казки,NaN,NaN,NaN,NaN,NaN,NaN
5,9789667047856,Найновіші пригоди Колька Колючки та Косі Вуханя,Всеволод Нестайко,4/9/09,160.0,uk,US,http://books.google.com/books/content?id=WtP9C...
6,966716134X,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,9789661200394,Незвичайні історії про Іванка,NaN,NaN,NaN,NaN,NaN,NaN
8,9789663190792,Королівство рожевих снів: Казки та легенди,Круковська Галина,NaN,NaN,NaN,NaN,NaN
9,9663190795,Королівство рожевих снів: Казки та легенди,Круковська Галина,NaN,NaN,NaN,NaN,NaN


### OpenLibrary enrishment attempt

In [44]:
openlibraryapi = "https://openlibrary.org/api/books?bibkeys=ISBN:{}"

for index, row in df.iterrows():
    isbn = row["ISBN Code"]
    resp = requests.get(openlibraryapi.format(isbn))
    payload = resp.text.replace("var _OLBookInfo = {", "{").replace("};", "}")
    if payload != "{}":
        parsed_payload = json.loads(payload)
        url = parsed_payload["ISBN:{}".format(isbn)]["preview_url"]
        print("we found potential update for ISBN {}, here are more details: {}".format(isbn, url))
    else:
        print("no updates found for ISBN {}".format(isbn))
print("done..")

no updates found for ISBN 9789663437354
no updates found for ISBN 9789664620397
no updates found for ISBN 9786176951223
no updates found for ISBN 9786177084180
no updates found for ISBN 9789664625453
no updates found for ISBN 9789667047856
no updates found for ISBN 966716134X
no updates found for ISBN 9789661200394
no updates found for ISBN 9789663190792
no updates found for ISBN 9663190795
no updates found for ISBN 9789661800907
no updates found for ISBN 9786176951889
no updates found for ISBN 9789662522082
no updates found for ISBN 9789661694629
no updates found for ISBN 9786176903109
we found potential update for ISBN 9789667047818, here are more details: https://openlibrary.org/books/OL27644524M/Povitrjani_ribki
no updates found for ISBN 9789664623473
no updates found for ISBN 9786177084692
no updates found for ISBN 9789669761057
no updates found for ISBN 9786175850275
no updates found for ISBN 9789662522075
no updates found for ISBN 9789669172075
no updates found for ISBN 97896642